# split data

In [17]:
import torch
import numpy as np
import json
import os
import cv2
import random
import glob
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

import tqdm
import json
import pickle
from shutil import copy

In [36]:
!mkdir dataset
!mkdir dataset/fold1
!mkdir dataset/fold1/images
!mkdir dataset/fold1/images/train
!mkdir dataset/fold1/images/val_R8
!mkdir dataset/fold1/images/val_R9
!mkdir dataset/fold1/images/val_R10
!mkdir dataset/fold1/images/val_R_other
!cp -r dataset/fold1/images dataset/fold1/labels

!cp -r dataset/fold1 dataset/fold2 
!cp -r dataset/fold1 dataset/fold3 
!cp -r dataset/fold1 dataset/fold4 
!cp -r dataset/fold1 dataset/fold5

# !mkdir logs


In [44]:
image_folder = '/home/hana/sonnh/kaggle-vin/dataset/images_only/train_jpg'   # thư mục image gốc 
csv_train_only_box = '/home/hana/sonnh/kaggle-vin/dataset/images_only/train_only_box.csv'
data_p_dir = 'data.p'
fold = 0 # fold ở đây là từ 0 -> 4  nhưng ở folder là từ 1 -> 5

## TRAIN

In [11]:
df = pd.read_csv(csv_train_only_box)
with open(data_p_dir, 'rb') as fp:
    data_ = pickle.load(fp)    

In [12]:
data = {}

for i in range(len(df)):
    image_id = df.iloc[i]['image_id']
    if image_id not in data:
        data[image_id] = {}
    rad_id = df.iloc[i]['rad_id']
    
    if rad_id not in data[image_id]:
        data[image_id][rad_id] = []
        
    if 'fold' not in data[image_id]:
        data[image_id]['fold'] = df.iloc[i]['fold']

    class_id = df.iloc[i]['class_id']
    if class_id != 14:
        width = data_['{}.dicom.png'.format(image_id)]['width']
        height = data_['{}.dicom.png'.format(image_id)]['height']
        xmax = df.iloc[i]['x_max']
        ymax = df.iloc[i]['y_max']
        xmin = df.iloc[i]['x_min']
        ymin = df.iloc[i]['y_min']

        x_center = ((xmax + xmin)//2) / width
        b_width = (xmax - xmin) / width
        
        y_center = ((ymax + ymin)//2) / height
        b_height = (ymax - ymin) / height
        if b_height == 0:
            print(ymax, ymin, height, image_id)
        
        data[image_id][rad_id].append([class_id, x_center, y_center, b_width, b_height])

In [45]:
for image_id in data:
    if data[image_id]['fold'] != fold:
        image_path = 'dataset/fold{}/images/train'.format(fold + 1)
        label_path = 'dataset/fold{}/labels/train'.format(fold + 1)
    else:
        continue
        
    for rad_id in data[image_id]:
        if rad_id == 'fold':continue
            
        f= open(os.path.join(label_path, '{}_{}.txt'.format(image_id, rad_id)),"w+")
            
        for anno in data[image_id][rad_id]:
            f.write("{} {} {} {} {}\n".format(anno[0], anno[1], anno[2], anno[3], anno[4]))
        
        f.close()
        copy('{}/{}.jpg'.format(image_folder, image_id), '{}/{}_{}.jpg'.format(image_path, image_id, rad_id))

In [22]:
!ls dataset/fold1/labels/train |wc -l #10545 ảnh là ok

10545


## val

In [37]:
for image_id in data:
    if data[image_id]['fold'] != fold:
        continue
    else:
        image_path_ = 'dataset/fold1/images/val_{}'
        label_path_ = 'dataset/fold1/labels/val_{}'
        
    for rad_id in data[image_id]:
        if rad_id == 'fold':continue
        if rad_id in ['R9', 'R8', 'R10']:
            rad = rad_id
        else:
            rad = 'R_other'
 
        image_path = image_path_.format(rad)
        label_path = label_path_.format(rad)

        f= open(os.path.join(label_path, '{}_{}.txt'.format(image_id, rad_id)),"w+")

        for anno in data[image_id][rad_id]:
            f.write("{} {} {} {} {}\n".format(anno[0], anno[1], anno[2], anno[3], anno[4]))

        f.close()
        copy('{}/{}.jpg'.format(image_folder, image_id), '{}/{}_{}.jpg'.format(image_path, image_id, rad_id))

In [41]:
!ls dataset/fold1/images/val_R_other|wc -l # mỗi r8 r9 r10 có 834 ảnh, other có 135 ảnh

135


# train

In [ ]:
nohup python3 -u train.py \
--img 1920 \
--batch 8 \
--epochs 50 \
--data vin_fold1.yaml \   # fix lại có này cho mỗi fold
--cfg yolov5x6_vin.yaml \
--hyp data/hyp.vin2.yaml \
--worker 16 \
--weight yolov5x6.pt \
--multi-scale > logs/log_fold1.log &  # nhớ thay đổi đưuòng dẫn log cho từng fold 

In [43]:
!python3 -u train.py \
--img 960 \
--batch 8 \
--epochs 50 \
--data vin_fold1.yaml \   
--cfg yolov5x6_vin.yaml \
--hyp data/hyp.vin2.yaml \
--worker 16 \
--weight yolov5x6.pt \
--multi-scale

SyntaxError: invalid syntax (<ipython-input-43-6931c4a2c638>, line 2)